# 一回でいい

In [90]:
# import
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import random
from ipywidgets import interact, interact_manual
import ipywidgets as widgets
from IPython import display
import time
from pathlib import Path
import pandas as pd
import os
import copy
import networkx as nx
import pickle
import sys
from collections import Counter
from scipy import stats
%matplotlib inline

# change directory

os.chdir("/home/kyoko/Develop/ActualCausation/MABE/MABE_contents/actual_agency")

# import packages -- pyphi

from pyanimats import *
from pyTPM import *
import actual_agency as agency
import pyphi
from pyphi import actual, config, Direction

pyphi.config.VALIDATE_SUBSYSTEM_STATES = False
pyphi.config.PARTITION_TYPE = 'TRI'

# Change directory back to Example directory, now in MABE_contents
os.chdir("../Experiments/analysis_now")

%load_ext autoreload
%autoreload 2
%aimport pyTPM, pyanimats, actual_agency

# make an array
datas = np.zeros(3000).reshape(250, 12)

prearr = np.sort(np.tile(np.arange(1,11), (1,5))).reshape(50, 1) 
runarr = np.tile(prearr.copy(), (5,1)) #Runs
hidarr = np.sort(np.tile(np.arange(2,7), (1,50))).reshape(250, 1) #Hidden nodes
stsarr = np.tile(np.arange(1,6), (1,50)).reshape(250, 1)

datas[:,0:1] = hidarr
datas[:,1:2] = runarr
datas[:,2:3] = stsarr

# defining some parameters to use 
generations = 30000
n_agents = int(generations/500.+1)
n_runs = 10
n_trials = 64
n_data = 5

agent = 60 # saved generation
trial = 60 # 0~63
run = 1 # 0 ~ n_runs-1 change only this


# Get Line-of-descent (LOD) data from MABE output
    
LOD_data = ['190822_7156331_h2_LOD_data.pkl','190822_7156331_h3_LOD_data.pkl','190822_7156331_h4_LOD_data.pkl','190822_7156331_h5_LOD_data.pkl','190822_7156331_h6_LOD_data.pkl']
LODs = []    
for i in range(len(LOD_data)):
    with open(LOD_data[i],'rb') as f:
        LODs.append(pickle.load(f))

# Get genome data
all_genomes = ['190822_7156331_h2_genome.pkl','190822_7156331_h3_genome.pkl','190822_7156331_h4_genome.pkl','190822_7156331_h5_genome.pkl','190822_7156331_h6_genome.pkl']
genomes = []
for i in range(len(all_genomes)):
    with open(all_genomes[i],'rb') as f:
        genomes.append(pickle.load(f))


# Get activity data
all_activity = ['190822_7156331_h2_activity.pkl','190822_7156331_h3_activity.pkl','190822_7156331_h4_activity.pkl','190822_7156331_h5_activity.pkl','190822_7156331_h6_activity.pkl']
activities = []
for i in range(len(all_activity)):
    with open(all_activity[i],'rb') as f:
        activities.append(pickle.load(f))


# Define functions to find states
# find the most frequent state


def counting3(data):
    c = Counter(data)
    freq_scores = c.most_common()
    if len(freq_scores) < 3:
        return(freq_scores)
    else:
        third_count = freq_scores[2][1]
    
        modes = [freq_scores[0][:], freq_scores[1][:]]
        for p in range(2,len(freq_scores)): #出現回数を比較、追加
            if freq_scores[p][1] == third_count and p < 5:
                modes.append(freq_scores[p])
        return(modes)

# change num(10) to bin(2)
def int2binary(n,node_n):
    holi = [int(x) for x in bin(n)[2:].zfill(node_n)]
    holi.reverse()
    return holi

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
# brain acivity, genome, TPM, TPM_gates, cm
genome = []
TPM, TPM_gates, cm = [],[],[]
n_connection = np.zeros(n_data*n_runs)
brain_activity = []

for data in range(n_data):
    n_hnodes = data+2
    for n in range(n_runs):
        place = 5*data + n
        
        # culculate brain activity
        brain_activity.append(agency.getBrainActivity(activities[data][n],n_agents=n_agents, n_nodes=n_hnodes+4, n_hidden=n_hnodes))
        #actarr = brain_activity[place][agent][trial]

        # genome, TPM, TPM_gate, cm
       
        genome.append(agency.get_genome(genomes[data], n, agent))
#print(len(genome[4]))

        a, b, c = genome2TPM(genome[place], n_nodes=n_hnodes+4, n_sensors=2, n_motors=2, gate_type='deterministic',states_convention='loli',remove_sensor_motor_effects=True)
        TPM.append(a)
        TPM_gates.append(b)
        cm.append(c) # right?
        


Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating activity matrix from MABE output...
Creating a

In [96]:
dec = []
label = [['S1','S2','M1','M2','H1','H2'],['S1','S2','M1','M2','H1','H2','H3'],['S1','S2','M1','M2','H1','H2','H3','H4'],['S1','S2','M1','M2','H1','H2','H3','H4','H5'],['S1','S2','M1','M2','H1','H2','H3','H4','H5','H6']]
state = []
network = []
prestate = []


for data in range(n_data):
    n_hnodes = data+2
    for n in range(n_runs):
        dec = []
        place = 10*data + n
        actarr = brain_activity[place][agent][trial]
        for j in range(len(brain_activity[place][0][0])): # 2進法 -> 10進法変換 行35,列node数の行列 
            num = 0
            for k in range(len(brain_activity[place][0][0][0])): # = hnode + 4
                num = num + (2 ** k) * actarr[j][k] 
                num = int(num) # 2進法への変換のためfloat -> int型
            dec.append(num)

#         c = Counter(dec)
        prestate.append(counting3(dec))
prestate # right?

[[(36, 11), (16, 8), (4, 8)],
 [(20, 24), (56, 4), (0, 1), (52, 1), (57, 1)],
 [(36, 25), (8, 4), (0, 1), (9, 1), (15, 1)],
 [(0, 6), (16, 6), (48, 5), (40, 5)],
 [(32, 17), (0, 3), (36, 3)],
 [(40, 15), (56, 3), (14, 3), (41, 3)],
 [(52, 14), (12, 13), (0, 1), (13, 1), (37, 1)],
 [(5, 27), (24, 3), (12, 2)],
 [(0, 15), (20, 14), (1, 1), (29, 1), (19, 1)],
 [(0, 14), (52, 14), (1, 1), (29, 1), (31, 1)],
 [(112, 10), (0, 3), (56, 3), (24, 3)],
 [(0, 9), (24, 8), (18, 3), (70, 3)],
 [(84, 9), (44, 8), (68, 6)],
 [(60, 7), (12, 6), (120, 3), (72, 3), (121, 3)],
 [(32, 14), (52, 13), (60, 2)],
 [(53, 22), (100, 3), (48, 2), (112, 2), (40, 2)],
 [(48, 14), (49, 3), (97, 3), (107, 3), (72, 3)],
 [(44, 14), (4, 14), (0, 1), (45, 1), (17, 1)],
 [(84, 21), (108, 7), (0, 1), (96, 1), (109, 1)],
 [(32, 15), (8, 4), (40, 3)],
 [(184, 9), (92, 9), (93, 3), (235, 3), (191, 3)],
 [(216, 13), (92, 8), (217, 2), (125, 2), (111, 2)],
 [(212, 9), (76, 8), (92, 3), (196, 3)],
 [(32, 9), (8, 8), (2, 5)],
 

In [95]:
len(prestate)


50

In [72]:
# fitness, connection, state, network
#dec = []
label = [['S1','S2','M1','M2','H1','H2'],['S1','S2','M1','M2','H1','H2','H3'],['S1','S2','M1','M2','H1','H2','H3','H4'],['S1','S2','M1','M2','H1','H2','H3','H4','H5'],['S1','S2','M1','M2','H1','H2','H3','H4','H5','H6']]
#state = []
network = []

for data in range(n_data):
    n_hnodes = data+2
    for n in range(n_runs):
        place = 5*data + n
        
        #fitness
    
        LODs[data][n]['fitness'] = (LODs[data][n]['correct_AVE']
                               /(LODs[data][n]['correct_AVE']+LODs[data][n]['incorrect_AVE']))
        #print(LODs[data][n]['fitness'][60])
        
        # culculate num of connection
        
        # for i in range(n_data*n_runs):
        #     n_connection[i] = np.count_nonzero(cm[i])
                
        #state
        
        
        
        
        for i in range(len(prestate)):
            state.append(int2binary(prestate[i][0],n_hnodes+4))

        # network
        network.append(pyphi.network.Network(np.array(TPM[place]), cm=np.array(cm[place]), 
                    purview_cache=None)) #node_labels=tuple(label[data]), 
       

TypeError: 'tuple' object cannot be interpreted as an integer

In [101]:
# ba = np.array(brain_activity)
# ba_shape, ba.type
dana = np.array(data)
dana

array(4)

In [75]:
# subsystem, phi
Phi = []
hnodes = [(4,5),(4,5,6),(4,5,6,7),(4,5,6,7,8),(4,5,6,7,8,9)]
for data in range(n_data):
    n_hnodes = data+2
    for n in range(n_runs):
        place = 5*data + n
        for i in range(len(state)):
            subsystem = pyphi.Subsystem(network[place], state[i], nodes=hnodes[data])
            Phi = pyphi.compute.phi(subsystem)
            print("Data %d, Run %d, state %d's phi is %d"%(data, n, i, Phi))
            

In [ ]:
n_connection = np.zeros(n_data*n_runs)
for i in range(50):
    n_connection[i] = np.count_nonzero(cm[i])
n_connection

In [81]:
honya = [0, 1, 2, 3] 

# frompyfunc 
def _body(x): 
    return x**2 
body = np.frompyfunc(_body, 1, 1) 

result = body(honya)
print(result)

[0 1 4 9]


In [ ]:
# 各計算結果をdatasに格納
# state(3列目)
for i in range(len(state)):
    if i < 5:
        datas[(n-1)*5 + i][3] = prestate[i][0]


for i in range()
np.set_printoptions(threshold = 10000)
datas

In [105]:
def counting3(data): 
    assert type(data) is np.ndarray, "Argument 'data' must be numpy.ndarray @counting3(data)" 
    u,c = np.unique(data, return_counts=True) 
    s = np.argsort(c)[::-1] 
    sc = c[s] 
    su = u[s] 
    l = (sc>=sc[2]).sum() 

    return su[0:l if l<6 else 5] 

In [107]:
dec = []
label = [['S1','S2','M1','M2','H1','H2'],['S1','S2','M1','M2','H1','H2','H3'],['S1','S2','M1','M2','H1','H2','H3','H4'],['S1','S2','M1','M2','H1','H2','H3','H4','H5'],['S1','S2','M1','M2','H1','H2','H3','H4','H5','H6']]
state = []
network = []
prestate = []

dana = np.array(data)

for dana in range(n_data):
    n_hnodes = data+2
    for n in range(n_runs):
        dec = []
        place = 10*data + n
        actarr = brain_activity[place][agent][trial]
        for j in range(len(brain_activity[place][0][0])): # 2進法 -> 10進法変換 行35,列node数の行列 
            num = 0
            for k in range(len(brain_activity[place][0][0][0])): # = hnode + 4
                num = num + (2 ** k) * actarr[j][k] 
                num = int(num) # 2進法への変換のためfloat -> int型
            dec.append(num)

        c = Counter(dec)
        prestate.append(counting3(dec))
prestate # right?

AssertionError: Argument 'data' must be numpy.ndarray @counting3(data)

In [87]:
# change num(10) to bin(2) 
def int2binary(n, node_n): 
    assert 2**node_n < n, "Number overflows array capacity 2^node_n" 
    return np.array(list(bin(n)[2:].zfill(node_n)), dtype=np.int32)[::-1]

In [108]:
genome = [] 
TPM, TPM_gates, cm = [],[],[] 
n_connection = np.zeros(n_data*n_runs) 
brain_activity = [] 

# Wrapper 
get_brain_activity = np.frompyfunc( 
    lambda d,n: agency.getBrainActivity(activities[d][n], n_agents=n_agents, n_nodes=(d+2)+4, n_hidden=(d+2)), 
    2, 1 
    ) 

get_genome = np.frompyfunc( 
    lambda d,n: agency.get_genome(genomes[d], n, agent), 
    2, 1 
    ) 

get_tpm_gates_cm = np.frompyfunc( 
    lambda d,n: np.array(genome2TPM(genome[5*d+n], 
    n_nodes=(d+2)+4, 
    n_sensors=2, 
    n_motors=2, 
    gate_type='deterministic', 
    states_convention='loli', 
    remove_sensor_motor_effects=True)), 
    2, 3 
    ) 

args = cartesian((np.arange(n_data), np.arange(n_runs))) 

brain_activity = get_brain_activity(args[:,0], args[:,1]) 
genome = get_genome(args[:,0], args[:,1]) 
TPM, TPM_gates, cm = get_tpm_gates_cm(args[:,0], args[:,1])


NameError: name 'cartesian' is not defined